### Timestamp

This notebook covers the basic of creating UDTFs for manipulating a `Timestamp`. For a list of supported timestamp precisions, check the [Heavy.AI docs page](https://docs.heavy.ai/sql/data-definition-ddl/datatypes-and-fixed-encoding#datatypes).

For the API, check the RBC ReadTheDocs page:
* [`Timestamp`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.Timestamp.html)

In [1]:
import warnings; warnings.filterwarnings('ignore')

### Connect to the HeavyDB server

In [2]:
# NBVAL_IGNORE_OUTPUT
from rbc.heavydb import RemoteHeavyDB
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

Timestamp requires HeavyDB 6.0 or newer

In [3]:
# NBVAL_IGNORE_OUTPUT
heavydb.version[:3]

(7, 0, 0)

### Load test data

In [4]:
from util import load_test_data
from rbc.tests import _TimestampTestTable
table_name = 'ts_table'
load_test_data(heavydb, _TimestampTestTable, table_name)

List of columns in the timestamp table

In [5]:
import pandas as pd
descr, result = heavydb.sql_execute(f'select * from {table_name}')
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

,t9,t9_2,t9_null,i8_2,t6
0,31539661001001001,1609462861001001001,6.584412e+16,1609462861001001001,31539661001001
1,65844122002002002,1643767322002002002,NaN,1643767322002002002,65844122002002
2,99975783003003003,1677812583003003003,2.117153e+18,1677812583003003003,99975783003003


### Define a function that operate on GeoMultiLineStrings

Function `to_polygon` takes a `Column<GeoMultiLineString>` as input and construct a polygon.

In [6]:
@heavydb("int32_t(TableFunctionManager, Column<Timestamp> ts, OutputColumn<int64_t> year, OutputColumn<int64_t> month, OutputColumn<int64_t> day)",
         devices=['cpu'])
def extract_year_month_day(mgr, ts, year, month, day):
    mgr.set_output_row_size(len(ts))
    for i in range(len(ts)):
        year[i] = ts[i].getYear()
        month[i] = ts[i].getMonth()
        day[i] = ts[i].getDay()
    return len(ts)

In [38]:
query = (f'''
    SELECT * FROM TABLE(extract_year_month_day(
        cursor(SELECT t9 from {table_name})
    ))
''')

descr, result = heavydb.sql_execute(query)
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

,year,month,day
0,1971,1,1
1,1972,2,2
2,1973,3,3


---------------------------

### (re)implement `generate_series`

HeavyDB ships with a function called `generate_series`, which as the name suggests, can generate a serie of timestamp values from `start_timestamp` to `end_timestamp`.

In [8]:
query = (f'''
SELECT
  *
FROM
  TABLE(
    extract_year_month_day(
      ts => CURSOR(
        SELECT
          *
        FROM
          TABLE(
            generate_series(
              TIMESTAMP(0) '2021-01-01 00:00:00',
              TIMESTAMP(0) '2021-09-04 00:00:00',
              INTERVAL '1' MONTH
            )
          )
      )
    )
  )
''')

descr, result = heavydb.sql_execute(query)
pd.DataFrame(list(result), columns=list(map(lambda x: x.name, descr)))

,year,month,day
0,2021,1,1
1,2021,2,1
2,2021,3,1
3,2021,4,1
4,2021,5,1
5,2021,6,1
6,2021,7,1
7,2021,8,1
8,2021,9,1


This same function can be reimplemented in RBC. Function `rbc_generate_series` below implements this notion of series in the same way as the builtin function `generate_series`.

In [9]:
@heavydb(
    "int32_t(TableFunctionManager, T start, T stop, K step, OutputColumn<T> generate_series)",
    T=["Timestamp"],
    K=["YearMonthTimeInterval", "DayTimeInterval"],
    devices=['cpu']
)
def rbc_generate_series(mgr, start, stop, step, series_output):
    if step.timeval == 0:
        return mgr.error_message("Timestamp division by zero")

    num_rows = step.numStepsBetween(start, stop) + 1
    if num_rows <= 0:
        mgr.set_output_row_size(0)
        return 0

    mgr.set_output_row_size(num_rows)

    for idx in range(num_rows):
        series_output[idx] = start + (step * idx)
    return num_rows

Let's also define a second helper function (`timestamp_extract`) to extract information from the resulting generated serie

In [19]:
@heavydb('int32(TableFunctionManager, Column<Timestamp> ts, TextEncodingNone text, OutputColumn<int64>)')
def timestamp_extract(mgr, inp, prop, out):
    size = len(inp)
    mgr.set_output_row_size(size)
    for i in range(size):
        if prop == 'Year':
            out[i] = inp[i].getYear()
        elif prop == 'Month':
            out[i] = inp[i].getMonth()
        elif prop == 'Day':
            out[i] = inp[i].getDay()
        elif prop == 'Hour':
            out[i] = inp[i].getHours()
        elif prop == 'Minute':
            out[i] = inp[i].getMinutes()
        else:
            out[i] = inp[i].getSeconds()
    return size

Query below generates a serie from `2021-01-01` to `2025-09-04` with step equals to 1 hour. The resulting dataframe contains the years in the generated serie:

In [35]:
query = (f'''
SELECT
  out0 as interval_
FROM
  TABLE(
    timestamp_extract(
      CURSOR(
        SELECT
          *
        FROM
          TABLE(
            generate_series(
              TIMESTAMP(0) '2021-01-01 00:00:00',
              TIMESTAMP(0) '2025-09-04 00:00:00',
              INTERVAL '1' HOUR
            )
          )
      ),
      'Year'
    )
  )
  group by interval_
''')

descr, result = heavydb.sql_execute(query)
pd.DataFrame(list(result), columns=list(map(lambda x: x.name, descr)))

,interval_
0,2021
1,2022
2,2023
3,2024
4,2025
